[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/arpm92/IAH_IA-Base/blob/main/gen_imagenes.ipynb)


# **Stable Diffusion** 🎨
*...using `🧨diffusers`*

Stable Diffusion es un modelo de difusión latente texto-imagen creado por los investigadores e ingenieros de [CompVis](https://github.com/CompVis), [Stability AI](https://stability.ai/) y [LAION](https://laion.ai/). Está entrenado con imágenes de 512x512 de un subconjunto de la base de datos [LAION-5B](https://laion.ai/blog/laion-5b/). Este modelo utiliza un codificador de texto congelado CLIP ViT-L/14 para condicionar el modelo a las indicaciones de texto. Con sus 860M de UNet y 123M de codificador de texto, el modelo es relativamente ligero y puede ejecutarse en muchas GPU de consumo.
Consulta la [ficha del modelo](https://huggingface.co/CompVis/stable-diffusion) para obtener más información.

Este cuaderno de Colab muestra cómo usar Stable Diffusion con la 🤗 Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers).

¡Vamos a empezar!

## 1. Cómo utilizar `StableDiffusionPipeline

Antes de sumergirnos en los aspectos teóricos de cómo funciona Stable Diffusion,
vamos a probarlo un poco 🤗.

En esta sección, mostramos cómo puede ejecutar la inferencia de texto a imagen ¡en sólo unas pocas líneas de código!

### Configuración

En primer lugar, por favor, asegúrese de que está utilizando un runtime GPU para ejecutar este cuaderno, por lo que la inferencia es mucho más rápida. Si el siguiente comando falla, utilice el menú `Runtime` de arriba y seleccione `Change runtime type`.

In [ ]:
!nvidia-smi

In [ ]:
# Install required packages: diffusers, transformers, and accelerate
!pip install diffusers transformers accelerate torch --quiet

# Install a specific version of the xformers package for performance improvements
!pip install xformers --quiet

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Load the Stable Diffusion model from the Hugging Face Model Hub
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# Move the pipeline to GPU for faster generation
pipe = pipe.to("cuda")

In [ ]:
# Function to create a grid of images
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols * w, rows * h))

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))

    return grid

In [ ]:
# Set the number of inference steps
num_inference_steps = 50
# Set the seed for reproducibility
generator = torch.manual_seed(42)  # Change seed for different results

In [ ]:
# Define grid size
num_cols = 2  # Number of columns in the grid
num_rows = 2  # Number of rows in the grid


In [ ]:
# Define the prompt for each image
prompt = ["a photograph of an astronaut riding a horse"] * num_cols


In [ ]:
all_images = []
for _ in range(num_rows):
    # Generate images with custom steps and seed
    images = pipe(
        prompt,
        num_inference_steps=num_inference_steps,  # Adjust the number of steps
        generator=generator  # Use the seed generator
    ).images
    all_images.extend(images)

In [ ]:
# Generate and display the grid of images
grid = image_grid(all_images, rows=num_rows, cols=num_cols)

# Display the image grid using matplotlib
plt.figure(figsize=(10, 10))
plt.imshow(grid)
plt.axis("off")  # Hide axes for clean image
plt.show()